In [7]:
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd


In [2]:
cred = credentials.Certificate("service-account.json")
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [10]:
mapping = {
 #   'user':'users_dev',
    'booking':'bookings_dev',
#    'emailReminders':'email_reminders_dev',
#    'smsReminders':'sms_reminders_dev'
}

In [11]:
for key in mapping:
    print(mapping[key])

bookings_dev


In [22]:
for key in mapping:
    temp = list(db.collection('{}'.format(key)).stream())
    temp_dict = list(map(lambda x: x.to_dict(), temp))
    globals()[f"df_{mapping[key]}"] = pd.DataFrame(temp_dict)
    globals()[f"df_{mapping[key]}"]['_imported_at'] = datetime.now()

In [23]:
df_bookings_dev.head()

,userId,date,className,cancelled,classDescription,classId,attendance,sessionId,notify,_imported_at
0,5370,2022-10-20 12:00,Toilet training,False,The first rule of toilet training is take your...,173248454,False,03961175-1ae3-4d13-8f93-f5c7b9ba3bb1,NaN,2022-12-01 11:06:15.631791
1,5370,2022-10-20 17:00,Training cues & tricks,True,Puppy won’t listen? Learn to use food and posi...,173248461,True,0d72ab7d-bb44-4477-847b-7d60d5fed669,NaN,2022-12-01 11:06:15.631791
2,5370,2022-11-14 17:00,First aid & health checks you can do,False,Poorly Puppy? Learn the signs of sickness and ...,173248449,True,15c92aee-57ec-4fe6-8ec3-5632e88f4ae7,False,2022-12-01 11:06:15.631791
3,5578,2022-12-08 12:00,Understanding body language,False,"Puppies can’t talk, but certainly have a lot t...",173248456,False,18e25659-8676-4409-9c63-81dcb7f8b082,NaN,2022-12-01 11:06:15.631791
4,5685,2022-12-08 12:00,Understanding body language,False,"Puppies can’t talk, but certainly have a lot t...",173248456,False,18e25659-8676-4409-9c63-81dcb7f8b082,NaN,2022-12-01 11:06:15.631791


In [20]:
key_path = r"C:\Users\nites\OneDrive\Documents\napo-nitesh-local-ae32-vpcservice-datawarehouse-879a160a28ee.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

job_config2 = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

In [21]:
dataset_name='firestore_dev'

for key in mapping:
    load_job = client.load_table_from_dataframe(
        globals()[f"df_{mapping[key]}"]
        ,"ae32-vpcservice-datawarehouse.{}.{}".format(dataset_name,mapping[key])
        ,job_config=job_config2
        ,location='EU'
    ) 
    print(load_job.result())